## Import required libraries:

In [2]:
import pandas as pd

## Load the excel data:

In [11]:
df = pd.read_excel('data/ntd/TS2.2TimeSeriesSysWideOpexpSvc_2.xlsx', sheet_name='UPT')
print(df)

      Last Report Year  NTD ID Legacy NTD ID  \
0               2017.0     1.0             1   
1               2017.0     2.0             2   
2               2017.0     3.0             3   
3               2017.0     4.0          0T04   
4               2017.0     5.0             5   
5               2017.0     6.0             6   
6               2017.0     7.0             7   
7               2017.0     8.0             8   
8               2017.0     9.0          0T09   
9               2017.0    10.0          0T10   
10              2017.0    11.0            11   
11              2017.0    12.0            12   
12              2016.0    13.0      TR27-001   
13              2017.0    14.0          0T14   
14              2017.0    15.0          0T15   
15              2017.0    16.0            16   
16              2017.0    17.0          0T17   
17              2017.0    18.0            18   
18              2017.0    19.0            19   
19              2017.0    20.0          